In [ ]:
import socket
import threading
import struct
import time
import cv2
import numpy
 
# 用于检测的类
class Carame_Accept_Object:
    def __init__(self, S_addr_port=("", 8880)):
        # 设置分辨率
        self.resolution = (640, 480)
        self.img_fps = 20  # 每秒传输帧数
        self.addr_port = S_addr_port
        self.Set_Socket(self.addr_port)
 
    # 设置socket
    def Set_Socket(self, S_addr_port):
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # 端口可复用
        self.server.bind(S_addr_port)
        self.server.listen(5)
        # print("the process work in the port:%d" % S_addr_port[1])
 
def check_option(object, client):
    # 按格式解码，确定帧数和分辨率
    info = struct.unpack('lhh', client.recv(8))
    if info[0] > 888:
        object.img_fps = int(info[0]) - 888  # 获取帧数
        object.resolution = list(object.resolution)
        # 获取分辨率
        object.resolution[0] = info[1]
        object.resolution[1] = info[2]
        object.resolution = tuple(object.resolution)
        return 1
    else:
        return 0
# 关键检测的函数 
def RT_Image(object, client, D_addr):
    if (check_option(object, client) == 0):
        return
    camera = cv2.VideoCapture(0)  # 从摄像头中获取视频
    img_param = [int(cv2.IMWRITE_JPEG_QUALITY), object.img_fps]  # 设置传送图像格式、帧数
    while (1):
        time.sleep(0.1)  # 推迟线程运行0.1s
        _, object.img = camera.read()  # 读取视频每一帧
 

        np_frame = object.img

        # 加载级联分类器
        # 加载opencv的接口，这里是face和eye的接口
        face_cascade = cv2.CascadeClassifier(
            '/home/xilinx/jupyter_notebooks/base/video/data/'
            'haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(
            '/home/xilinx/jupyter_notebooks/base/video/data/'
            'haarcascade_eye_tree_eyeglasses.xml')
 
        # 将图像转换为灰度图
        gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
        # 使用级联分类器检测人脸
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
 
        # 遍历检测到的人脸
        for (x, y, w, h) in faces:
            # 在原图像上绘制矩形框
            cv2.rectangle(np_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            # 获取人脸区域
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = np_frame[y:y + h, x:x + w]
 
            # 使用级联分类器检测眼睛
            eyes = eye_cascade.detectMultiScale(roi_gray)
            # 遍历检测到的眼睛
            for (ex, ey, ew, eh) in eyes:
                # 在人脸区域上绘制矩形框
               cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
        object.img = cv2.resize(object.img, object.resolution)  # 按要求调整图像大小(resolution必须为元组)
        _, img_encode = cv2.imencode('.jpg', object.img, img_param)  # 按格式生成图片
        img_code = numpy.array(img_encode)  # 转换成矩阵
        object.img_data = img_code.tostring()  # 生成相应的字符串
        try:

            client.send(
                struct.pack("lhh", len(object.img_data), object.resolution[0], object.resolution[1]) + object.img_data)
        except:
            camera.release()  
            return
 
if __name__ == '__main__':
    camera = Carame_Accept_Object()
    # 循环接收
    while (1):
        client, D_addr = camera.server.accept()
        clientThread = threading.Thread(None, target=RT_Image, args=(camera, client, D_addr,))
        clientThread.start()

In [2]:
import os
print(os.path.abspath('.'))

/home/xilinx/jupyter_notebooks


In [ ]:
!pip install pandas

  Using cached pandas-1.1.5.tar.gz (5.2 MB)
  Installing build dependencies ... -